<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#成績一本通（作業二）
* 目標：從 Sheet 讀各次作業/測驗 → 計平時/期末/總成績 → 回寫成績總表。
* AI 點子：針對「開放式作答」欄位，用模型產出整體摘要與常見迷思整理（不評分，只做總結）。
* google sheet:https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing

In [66]:
# 1. 安裝 Google GenAI SDK
!pip install google-genai

In [67]:
import os
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd
import datetime
#from types import new_class
import gradio as gr

creds,_ = default()
gc = gspread.authorize(creds)
gsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing')
worksheet = gsheet.worksheet('工作表1')
TOTAL_CELL = 'G2'

try:
    analysis_worksheet = gsheet.worksheet('AI 分析摘要')
except gspread.WorksheetNotFound:
    # 如果工作表不存在，則創建它
    analysis_worksheet = gsheet.add_worksheet(title='AI 分析摘要', rows="100", cols="20")
    # 可以寫入標題
    analysis_worksheet.update_cell(1, 1, "AI 分析時間")
    analysis_worksheet.update_cell(1, 2, "AI 關鍵摘要 1")
    analysis_worksheet.update_cell(1, 3, "AI 關鍵摘要 2")
    analysis_worksheet.update_cell(1, 4, "AI 關鍵摘要 3")

In [68]:
from google import genai
from google.genai import types

# 初始化 Gemini 客戶端

# 優先嘗試從環境變數中讀取 API 金鑰
api_key_from_env = os.environ.get('GEMINI_API_KEY')

try:
    if api_key_from_env:
        # 如果環境變數存在，則使用它來初始化客戶端
        client = genai.Client(api_key=api_key_from_env)
        print("✅ Gemini API 客戶端初始化成功 (使用環境變數)。")
    else:
        # 如果環境變數不存在，則拋出錯誤提醒使用者設定
        raise ValueError("環境變數 'GEMINI_API_KEY' 未設定。")

except Exception as e:
    # 這將捕獲到沒有金鑰或初始化失敗的錯誤
    print(f"❌ Gemini API 客戶端初始化失敗：{e}")
    client = None

✅ Gemini API 客戶端初始化成功 (使用環境變數)。


In [69]:
def get_total_amount_value():
    """
    從 Google Sheets 中讀取所有資料，計算金額總計並回傳【標準 Python 數字】。
    """
    try:
        # 讀取工作表的所有記錄（不包含標題行）
        df = pd.DataFrame(worksheet.get_all_records())

        if '金額' in df.columns:
            # 強制將總和轉換為標準 Python float，解決 int64 的問題
            total_money = float(df['金額'].sum(skipna=True))
            return total_money
        else:
            print("錯誤：找不到 '金額' 欄位。")
            return 0.0
    except Exception as e:
        print(f"總計計算失敗：{e}")
        return 0.0

In [70]:
def update_total_in_sheet():
    """
    計算總計，並將結果寫回 Google Sheets 的指定儲存格 (G1)。
    """
    total_value = get_total_amount_value()

    # 寫入 Google Sheet (使用 G 欄的第二行，即 G2)
    worksheet.update_cell(2, 7, total_value)

    # 回傳【格式化字串】給 Gradio 介面顯示
    return f"💰 總支出：{total_value:.2f} 元"

In [71]:
def populate_total_amount(amount_state):
    """
    從 gr.State 中讀取金額並填充到輸入框。
    """
    return amount_state

In [72]:
def write_to_sheet(date_str, time_str, item, money, category, pay_way):
    """
    接收使用者輸入，並寫入 Google Sheets 的函式。
    """
    try:
        # 驗證日期和時間格式
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        datetime.datetime.strptime(time_str, '%H:%M')

        # 確保金額是數字
        money = int(money)

        # 建立一個 DataFrame 來儲存單筆資料
        data = pd.DataFrame([{
            '日期': date_str,
            '時間': time_str,
            '品項': item,
            '金額': money,
            '類別': category,
            '支付方式': pay_way
        }])

        # 將 DataFrame 轉為列表以便寫入
        data_to_write = data.values.tolist()

        # 寫入資料到 Google Sheets
        worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')

        # 寫入成功後，【呼叫更新總計的函式】來更新 Sheet 和 Gradio 介面
        new_total_output = update_total_in_sheet()

        return "✅ 資料已成功寫入！", new_total_output, money


    except ValueError as ve:
        # 處理格式錯誤
        current_total_output = update_total_in_sheet()
        return f"❌ 格式錯誤：{ve}，請確認日期（YYYY-MM-DD）、時間（HH:MM）或金額的格式是否正確。", current_total_output, None
    except Exception as e:
        # 處理其他錯誤
        current_total_output = update_total_in_sheet()
        return f"❌ 寫入失敗：{e}", current_total_output, None

In [73]:
def split_bill(total_amount, num_people):
    """
    分帳計算函式。
    """
    try:
        total_amount = float(total_amount)
        num_people = int(num_people)

        if num_people <= 0:
            return "❌ 人數必須大於 0。"

        per_person_amount = total_amount / num_people
        return f"✅ 每人應付金額：{per_person_amount:.2f} 元"

    except ValueError:
        return "❌ 請確認金額和人數都為數字。"
    except Exception as e:
        return f"❌ 計算失敗：{e}"

In [74]:
def get_expense_data_for_analysis():
    """
    從 Google Sheets 讀取所有支出記錄並格式化為 Markdown 表格字串。
    """
    try:
        # 讀取工作表的所有記錄
        df = pd.DataFrame(worksheet.get_all_records())

        if df.empty:
            return "無支出記錄可供分析。"

        # 選擇相關欄位並轉換為字串，以便模型可以理解
        analysis_df = df[['日期', '品項', '金額', '類別', '支付方式']]

        # 將 DataFrame 轉換為 Markdown 表格字串
        data_markdown = analysis_df.to_markdown(index=False)
        return data_markdown

    except Exception as e:
        return f"讀取資料失敗: {e}"

In [75]:
def write_analysis_to_sheet(summaries):
    """
    將 AI 生成的摘要寫入 'AI 分析摘要' 工作表。
    summaries 應該是一個包含 3 個字串的列表。
    """
    # ... [此處程式碼與您上一次提交的 write_analysis_to_sheet 函式保持一致] ...
    # 這裡假設該函式能夠成功將 [時間, 摘要1, 摘要2, 摘要3] 寫入新的工作表
    try:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        data_to_write = [current_time] + summaries
        analysis_worksheet.append_row(values=data_to_write, value_input_option='USER_ENTERED')
        return "✅ AI 分析摘要已成功寫入 'AI 分析摘要' 工作表。"
    except Exception as e:
        return f"❌ 寫入 AI 摘要到試算表失敗: {e}"

In [76]:
def summarize_and_write_expenses():
    """
    呼叫 Gemini API 取得【3 點關鍵摘要】，將其寫入試算表，並回傳格式化輸出。
    """
    if client is None:
        return "❌ 缺少API金鑰！請設定環境變數 **GEMINI_API_KEY** 後，再使用此頁面。"

    expense_data = get_expense_data_for_analysis()
    if "無支出記錄" in expense_data or "讀取資料失敗" in expense_data:
        return expense_data

    # 使用嚴格的提示，確保回傳 3 行摘要
    prompt = f"""
    請擔任財務分析師，根據以下的支出數據進行分析。數據格式為 Markdown 表格：{expense_data}
    您的回答**必須**只包含以下三點，每一點為一行，並且不要包含任何其他解釋或抬頭：
    1. **消費結構最突出的洞察** (例如：外食佔總支出 60%)。
    2. **支付方式最顯著的趨勢** (例如：信用卡使用率達 85%，建議保持)。
    3. **一個具體的節省建議或警示** (例如：交通費異常偏高，考慮月票)。
    """

    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )

        raw_text = response.text.strip()
        summaries = [s.strip() for s in raw_text.split('\n') if s.strip()][:3]

        if len(summaries) < 3:
            return f"❌ AI 回覆格式不正確，只取得 {len(summaries)} 點摘要。原始回覆：\n\n{raw_text}"

        # **執行寫入試算表**
        write_status = write_analysis_to_sheet(summaries)

        # 構造回傳給 Gradio 的完整 Markdown 輸出
        return f"""
        {write_status}

        ---

        ### 🤖 AI 關鍵洞察 (已儲存至試算表)

        1. **消費結構：** {summaries[0]}
        2. **支付趨勢：** {summaries[1]}
        3. **節省建議：** {summaries[2]}
        """

    except Exception as e:
        return f"❌ Gemini API 或處理失敗：{e}"

In [77]:
def detailed_analyze_expenses():
    """
    呼叫 Gemini API 取得【詳細分析報告】，僅供 Gradio 介面顯示。
    """
    if client is None:
        return "❌ 缺少API金鑰！請設定環境變數 **GEMINI_API_KEY** 後，再使用此頁面。"

    expense_data = get_expense_data_for_analysis()
    if "無支出記錄" in expense_data or "讀取資料失敗" in expense_data:
        return expense_data

    # 使用開放式提示，鼓勵模型提供長篇、詳細的分析
    prompt = f"""
    請擔任財務分析師，根據以下的支出數據進行詳盡的分析和報告。
    數據格式為 Markdown 表格：{expense_data}

    請提供一份結構化的報告，包含：
    1. **支出總覽與結構細分**：指出主要支出類別和趨勢。
    2. **異常值或亮點**：是否有任何單筆高額支出或特別值得注意的消費行為。
    3. **未來預算和行為調整建議**：提供具體且可執行的財務管理建議。
    請使用 Markdown 格式並確保報告內容豐富，長度建議超過 300 字。
    """

    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )

        # 直接回傳模型的完整報告
        return response.text

    except Exception as e:
        return f"❌ Gemini API 呼叫失敗：{e}"

In [78]:
# --- Gradio 介面定義 ---
with gr.Blocks(title="chi_支出記錄器") as demo:
    gr.Markdown("## 📋 Google Sheets 支出記錄器")

    # 這裡新增一個 gr.State 來儲存金額
    amount_state = gr.State(value=None)

    with gr.Tabs():
        # 第一個分頁：支出記錄器
        with gr.TabItem("💸 支出記錄器"):
            gr.Markdown("請在下方輸入支出資料，資料將會自動寫入 Google Sheets。")

            with gr.Row():
                date_in = gr.Textbox(label="日期", placeholder="YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
                time_in = gr.Textbox(label="時間", placeholder="HH:MM", value=datetime.datetime.now().strftime("%H:%M"))

            with gr.Row():
                item_in = gr.Textbox(label="品項", placeholder="例如：午餐、車票")
                money_in = gr.Textbox(label="金額", placeholder="數字")

            with gr.Row():
                category_in = gr.Textbox(label="類別", placeholder="例如：外食、交通")
                pay_way_in = gr.Textbox(label="支付方式", placeholder="例如：現金、信用卡")

            submit_btn = gr.Button("寫入資料，並計算新總金額")

            with gr.Row():
                output_msg = gr.Markdown()
                # 介面載入時，直接呼叫 update_total_in_sheet() 來顯示當前總計
                total_output = gr.Markdown(value=update_total_in_sheet())

            # submit_btn.click 的設定保持不變，因為 write_to_sheet 已經更新了邏輯
            submit_btn.click(
                fn=write_to_sheet,
                inputs=[date_in, time_in, item_in, money_in, category_in, pay_way_in],
                outputs=[output_msg, total_output, amount_state]
            )

        # 第二個分頁：分帳小工具
        with gr.TabItem("💰 分帳小工具"):
            gr.Markdown("請輸入總金額和人數，按下按鈕即可計算每人應付金額。")

            with gr.Row():
                total_value = gr.Textbox(label="總金額", placeholder="例如：1000")
                num_people_in = gr.Textbox(label="人數", placeholder="例如：4")

            # 新增一個按鈕來從支出頁面取得金額
            fill_btn = gr.Button("讀取最後一筆支出金額")
            split_btn = gr.Button("計算分帳")

            split_output = gr.Markdown()

            # 設定點擊事件
            fill_btn.click(
                fn=populate_total_amount,
                inputs=amount_state,
                outputs=total_value
            )

            split_btn.click(
                fn=split_bill,
                inputs=[total_value, num_people_in],
                outputs=split_output
            )

        # 第三個分頁：AI 支出分析
        with gr.TabItem("🧠 AI 支出分析"):
            gr.Markdown("### 🤖 支出 AI 分析工具")
            gr.Markdown("請選擇您想要的分析深度。**精簡摘要**會將 3 點關鍵洞察儲存到 Google Sheet。")

            # 放置兩個按鈕在同一行
            with gr.Row():
                summarize_btn = gr.Button("1. 精簡摘要 (儲存至試算表)")
                detail_btn = gr.Button("2. 詳細報告 (僅顯示於此)")

            # 用於顯示 AI 分析結果的元件
            analysis_output = gr.Markdown("點擊上方按鈕開始分析，結果將顯示在此。")

            # 連接按鈕與對應的函式
            summarize_btn.click(
                fn=summarize_and_write_expenses,
                inputs=None,
                outputs=analysis_output
            )

            detail_btn.click(
                fn=detailed_analyze_expenses,
                inputs=None,
                outputs=analysis_output
            )

In [79]:
demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>